In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-07-31 00:31:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2022-07-31 00:31:18 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Stock_Market_Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# url="https://myclassbucket-01.s3.amazonaws.com/user_data.csv"
# spark.sparkContext.addFile(url)
# user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
# user_data_df.show()

In [ ]:
# url="https://myclassbucket-01.s3.amazonaws.com/user_payment.csv"
# spark.sparkContext.addFile(url)
# user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

# # Show DataFrame
# user_payment_df.show()

In [ ]:
# # Join the two DataFrame
# joined_df= user_data_df.join(user_payment_df, on="username", how="inner")
# joined_df.show()

In [ ]:
# # Drop null values
# dropna_df = joined_df.dropna()
# dropna_df.show()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# # Filter for only columns with active users
# cleaned_df = dropna_df.filter(col("active_user")  == True)
# cleaned_df.show()

In [ ]:
# # Create user dataframe to match active_user table
# clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])
# clean_user_df.show()

In [ ]:
# # Create user dataframe to match billing_info table
# clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"])
# clean_billing_df.show()

In [ ]:
# # Create user dataframe to match payment_info table
# clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"])
# clean_payment_df.show()

Postgres Setup

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://myclassdb-01.cd80yqfjmzyg.us-east-1.rds.amazonaws.com:5432/stock_market"
config = {"user":"postgres", 
          "password": "vazji6-jaqkux-sumqYb", 
          "driver":"org.postgresql.Driver"}


In [7]:
stock_df = spark.read.jdbc(url=jdbc_url, table='stocks', properties=config)

In [8]:
stock_df.createOrReplaceTempView("stocks")

In [12]:
st = stock_df.filter(stock_df['ticker'] == 'AAPL')

In [13]:
st.show()

+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|ticker|      date|           code|                open|                high|                 low|               close|            adjclose|              volume|
+------+----------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  AAPL|1985-03-18|AAPL_1985-03-18|0.102121002972126000|0.103237003087997400|0.102121002972126000|0.102121002972126000|0.079707682132720900|124768000.0000000...|
|  AAPL|1985-03-19|AAPL_1985-03-19|0.102121002972126000|0.103237003087997400|0.098214000463485700|0.098214000463485700|0.076658204197883600|171449600.0000000...|
|  AAPL|1985-03-20|AAPL_1985-03-20|0.099330000579357100|0.101003997027873900|0.099330000579357100|0.099330000579357100|0.077529251575469900|404969600.0000000...|
|  AAPL|1985-03-21|AAPL_1985

In [ ]:
# stock_df.show()

In [9]:
companies_df = spark.read.jdbc(url=jdbc_url, table='tickers', properties=config)

In [10]:
companies_df.show()

+---+------+--------------------+--------------------+--------------------+--------------------+----------------+-------+-----------+
| id|symbol|            security|         gics_sector| gics_sub_industries|        headquarters|date_first_added|    cik|    founded|
+---+------+--------------------+--------------------+--------------------+--------------------+----------------+-------+-----------+
|  1|   MMM|                  3M|         Industrials|Industrial Conglo...|Saint Paul, Minne...|      1976-08-09|  66740|       1902|
|  2|   AOS|         A. O. Smith|         Industrials|   Building Products|Milwaukee, Wisconsin|      2017-07-26|  91142|       1916|
|  3|   ABT|              Abbott|         Health Care|Health Care Equip...|North Chicago, Il...|      1964-03-31|   1800|       1888|
|  4|  ABBV|              AbbVie|         Health Care|     Pharmaceuticals|North Chicago, Il...|      2012-12-31|1551152|2013 (1888)|
|  5|  ABMD|             Abiomed|         Health Care|Health C

In [ ]:
stock_df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- date: date (nullable = true)
 |-- code: string (nullable = true)
 |-- open: decimal(38,18) (nullable = true)
 |-- high: decimal(38,18) (nullable = true)
 |-- low: decimal(38,18) (nullable = true)
 |-- close: decimal(38,18) (nullable = true)
 |-- adjclose: decimal(38,18) (nullable = true)
 |-- volume: decimal(38,18) (nullable = true)



In [ ]:
companies_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- symbol: string (nullable = true)
 |-- security: string (nullable = true)
 |-- gics_sector: string (nullable = true)
 |-- gics_sub_industries: string (nullable = true)
 |-- headquarters: string (nullable = true)
 |-- date_first_added: string (nullable = true)
 |-- cik: integer (nullable = true)
 |-- founded: string (nullable = true)



In [ ]:
df = spark.sql("SELECT ticker, date, code, adjclose, volume FROM stocks")

In [ ]:
df.show()

+------+----------+--------------+--------------------+--------------------+
|ticker|      date|          code|            adjclose|              volume|
+------+----------+--------------+--------------------+--------------------+
|   MOS|1989-11-02|MOS_1989-11-02|10.94354820251464...|142800.0000000000...|
|   MOS|1989-11-03|MOS_1989-11-03|10.94354820251464...|22000.00000000000...|
|   MOS|1989-11-06|MOS_1989-11-06|10.90209579467773...|224800.0000000000...|
|   MOS|1989-11-07|MOS_1989-11-07|10.77773571014404...|671600.0000000000...|
|   MOS|1989-11-08|MOS_1989-11-08|10.86064243316650...|63800.00000000000...|
|   MOS|1989-11-09|MOS_1989-11-09|10.73628330230712...|54000.00000000000...|
|   MOS|1989-11-10|MOS_1989-11-10|10.69482898712158...|353800.0000000000...|
|   MOS|1989-11-13|MOS_1989-11-13|10.69482898712158...|79600.00000000000...|
|   MOS|1989-11-14|MOS_1989-11-14|10.69482898712158...|21800.00000000000...|
|   MOS|1989-11-15|MOS_1989-11-15|10.65338325500488...|76200.00000000000...|

In [23]:
import pandas as pg
import numpy as np

In [21]:
df2=stock_df.filter( (stock_df["date"] >= '2001-01-01') | (stock_df['date'] <= '2022-07-15'))

In [22]:
df2.show()

Py4JJavaError: ignored